In [65]:
import pandas as pd
import numpy as np
import json
from pydantic import parse_obj_as
from fastapi.encoders import jsonable_encoder

from src.ml.core import encoder
from src.ml.models import Participant

from src.ml.service.preprocessing import download_mongo_participants

In [66]:
data = await download_mongo_participants()
data_json = jsonable_encoder(data)

data_f = pd.read_json(json.dumps(data_json))  

In [67]:
data_f['champion_name'] = encoder().fit_transform(data_f['champion_name'])
data_f['lane'] = encoder().fit_transform(data_f['lane'])
data_f['win'] = data_f['win'].astype(int)

item_encoder = encoder()
unique_item_ids = sorted(list(set(x for item_list in data_f['items'] for x in item_list)))
item_encoder.fit(unique_item_ids)
data_f['items'] = data_f['items'].apply(
    lambda x: item_encoder.transform(x).tolist()
)
data_f['items'] = data_f['items'].apply(np.array)

In [70]:
# Нормализация данных для LSTM
columns_to_norm = data_f.columns.difference(['items'])
data_f_normalized = data_f[columns_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

items_min = data_f['items'].apply(np.min).min()
items_max = data_f['items'].apply(np.max).max()
items_normalized = (data_f['items'] - items_min) / (items_max - items_min) 

# data_f_normalized['items'] = items_normalized

In [76]:
# TO torch tensor
import torch
from torch.utils.data import DataLoader, random_split

import random

random.seed(42)

base_fields_tensor = torch.tensor(data_f_normalized.values, dtype=torch.float32)

items_list = items_normalized.tolist()
items_tensor = torch.tensor(items_list, dtype=torch.float32)

data_tensor = torch.cat((base_fields_tensor, items_tensor), dim=1)

# Разделение данных на обучающую и тестовую выборки
train_size = int(0.8 * len(data_tensor))
test_size = len(data_tensor) - train_size

train_data, test_data = random_split(data_tensor, [train_size, test_size])

# Создание DataLoader для обучения и тестирования
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
# Создание модели
...

In [81]:
# Настройка модели
print("""
      ['В python pandas я занимаюсь препроцессингом данных для модели lstm. В моих данных есть поле, содержащее списки одинакового размера, наполненные числами. Я преобразовал массивы в этом поле к numpy.array, и теперь когда я пытаюсь сделать нормализацию \ndata_f_normalized = (data_f - data_f.min()) / (data_f.max() - data_f.min())\nМне выдает ошибку  The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()', "Ошибка возникает из-за того, что операции min() и max() возвращают массивы, а не скалярные значения, когда они применяются к столбцу с массивами numpy. Вам нужно найти минимальное и максимальное значения по всем элементам во всех массивах этого столбца.\n\nВот пример, как это можно сделать:\n\n``` python\nimport pandas as pd\nimport numpy as np\n\n# Создаем пример данных\ndata = pd.DataFrame({'A': [np.array([1, 2, 3]), np.array([4, 5, 6]), np.array([7, 8, 9])]})\n\n# Находим минимальное и максимальное значения по всем элементам массивов\nmin_value = data['A'].apply(np.min).min()\nmax_value = data['A'].apply(np.max).max()\n\n# Нормализуем данные\ndata['A_normalized'] = data['A'].apply(lambda x: (x - min_value) / (max_value - min_value))\n\nprint(data)\n```\n\nЭтот код создает пример данных с одним столбцом, содержащим массивы numpy, находит минимальное и максимальное значения по всем элементам массивов, а затем нормализует данные и добавляет их в новый столбец 'A_normalized'.", 'Но я же оперирую со всем датасетом а не с отдельными столбцами, т.е. когда я выполняю функцию data_f_normalized = (data_f - data_f.min()) / (data_f.max() - data_f.min()) все поля кроме поля items, которое содержит списки. Так что мне нужно нормализовать весь дата фрейм. Как поступить?', "В таком случае, вы можете нормализовать все столбцы, кроме столбца 'items', а затем объединить нормализованные столбцы с исходным столбцом 'items'. Вот пример, как это можно сделать:\n\n``` python\nimport pandas as pd\nimport numpy as np\n\n# Создаем пример данных\ndata = pd.DataFrame({'A': [1, 4, 7], 'B': [2, 5, 8], 'items': [np.array([1, 2, 3]), np.array([4, 5, 6]), np.array([7, 8, 9])]})\n\n# Выбираем все столбцы, кроме 'items'\ncolumns_to_normalize = data.columns.difference(['items'])\n\n# Нормализуем выбранные столбцы\ndata_normalized = data[columns_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))\n\n# Добавляем столбец 'items' обратно в нормализованный датафрейм\ndata_normalized['items'] = data['items']\n\nprint(data_normalized)\n```\n\nЭтот код создает пример данных с тремя столбцами, где столбец 'items' содержит массивы numpy. Затем мы выбираем все столбцы, кроме 'items', и нормализуем их. После этого мы добавляем столбец 'items' обратно в нормализованный датафрейм.", 'Отлично, я провел препроцессинг данных. В итоге у меня есть дата фрейм с несколькими обычными полями heroName, win, Lane и полем items, где лежат списки. Все поля я привел в числовой формат, нормализовал для модели lstm. \nКак теперь обучить модель? Начнем с преобразования этого датафрейма в тензор, если преобразовать его обычным способом\ndata_tensor = torch.tensor(data_f_normalized.values, dtype=torch.float32)\nТо мне выдает ошибку, из-за наличия в полях numpy array (поля items)', "Для обучения модели LSTM на данных с несколькими полями, вам нужно преобразовать датафрейм в тензоры, которые могут быть использованы в модели. Поскольку у вас есть поле со списками, вы не можете напрямую преобразовать датафрейм в тензор. Вам нужно отдельно обработать столбец 'items' и остальные столбцы.\n\nВот пример, как это можно сделать:\n\n``` python\nimport pandas as pd\nimport numpy as np\nimport torch\n\n# Создаем пример данных\ndata = pd.DataFrame({'heroName': [1, 4, 7], 'win': [0, 1, 0], 'Lane': [2, 5, 8], 'items': [np.array([1, 2, 3]), np.array([4, 5, 6]), np.array([7, 8, 9])]})\n\n# Выбираем все столбцы, кроме 'items'\ncolumns_without_items = data.columns.difference(['items'])\n\n# Преобразуем остальные столбцы в тензор\ndata_tensor_without_items = torch.tensor(data[columns_without_items].values, dtype=torch.float32)\n\n# Преобразуем столбец 'items' в тензор\nitems_list = data['items'].tolist()\nitems_tensor = torch.tensor(items_list, dtype=torch.float32)\n\n# Объединяем тензоры в один тензор\ndata_tensor = torch.cat((data_tensor_without_items, items_tensor), dim=1)\n\nprint(data_tensor)\n```\n\nЭтот код создает пример данных с четырьмя столбцами, где столбец 'items' содержит массивы numpy. Затем мы выбираем все столбцы, кроме 'items', и преобразуем их в тензор. После этого мы преобразуем столбец 'items' в тензор и объединяем оба тензора в один.\n\nТеперь у вас есть тензор `data_tensor`, который можно использовать для обучения модели LSTM. Обратите внимание, что вам, возможно, потребуется изменить форму тензора в соответствии с ожидаемым входом модели LSTM (обычно требуется 3D-тензор с формой `[batch_size, sequence_length, input_size]`).", 'Ок, мы закончили препроцессинг, давай перейдем к созданию и обучению модели.\nПредметная область - игра League of Legends, поля\nheroName - имя героя, lane - позиция героя, win - победа или поражения, items - список предметов, максимум 7\nЦель -  по входному списку предметов (от 0 до 6), имени героя и его позиции выдать рекомендацию по покупке следующего предмета, который обычно собирается и который вероятней приведет его к победе (для этого в датасете поле win), или же нужно это поле использовать так, чтобы записи где герой проиграл имели меньший вес - тут я не знаю, решай сам.', "Для решения этой задачи мы будем использовать многослойную LSTM сеть. В качестве входных данных будем использовать имя героя, позицию героя и список предметов. Выходом сети будет вероятность того, что следующий предмет приведет к победе. В качестве функции потерь будем использовать Binary Cross Entropy Loss, т.к. у нас есть только два возможных исхода - победа или поражение.\n\nВот пример кода для создания и обучения модели:\n\n```python\nimport torch\nimport torch.nn as nn\nimport torch.optim as optim\n\n# Создаем LSTM модель\nclass LSTMModel(nn.Module):\n    def __init__(self, input_size, hidden_size, num_layers, output_size):\n        super(LSTMModel, self).__init__()\n        self.hidden_size = hidden_size\n        self.num_layers = num_layers\n        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)\n        self.fc = nn.Linear(hidden_size, output_size)\n\n    def forward(self, x):\n        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()\n        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()\n        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))\n        out = self.fc(out[:, -1, :])\n        return out\n\n# Гиперпараметры\ninput_size = data_tensor.shape[1]\nhidden_size = 64\nnum_layers = 2\noutput_size = 1\nnum_epochs = 100\nlearning_rate = 0.001\n\n# Инициализация модели, функции потерь и оптимизатора\nmodel = LSTMModel(input_size, hidden_size, num_layers, output_size)\ncriterion = nn.BCEWithLogitsLoss()\noptimizer = optim.Adam(model.parameters(), lr=learning_rate)\n\n# Обучение модели\nfor epoch in range(num_epochs):\n    # Преобразуем данные в нужную форму\n    inputs = data_tensor.unsqueeze(1)  # Добавляем размерность sequence_length\n    targets = torch.tensor(data['win'].values, dtype=torch.float32).view(-1, 1)\n\n    # Forward pass\n    outputs = model(inputs)\n    loss = criterion(outputs, targets)\n\n    # Backward and optimize\n    optimizer.zero_grad()\n    loss.backward()\n    optimizer.step()\n\n    if (epoch + 1) % 10 == 0:\n        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')\n```\n\nПосле обучения модели, вы можете использовать ее для предсказания вероятности победы на основе имени героя, позиции и списка предметов. Просто передайте соответствующий тензор с данными в модель, и она выдаст вероятность победы.\n\nОбратите внимание, что этот код является примером и может потребовать дополнительной настройки и оптимизации для вашей конкретной задачи. Вам также может потребоваться разделить ваши данные на обучающую и тестовую выборки для оценки производительности модели.", 'Нет, главная функция - это рекомендация предметов. Т.е. на вход подается например items: [2312, 234, 0, 0, 0, 0, 0] и сервис должен выдать предмет, который нужно собрать следующим.', "Понял, в таком случае, можно использовать классификационную модель, которая будет предсказывать следующий предмет на основе текущих предметов, имени героя и его позиции. Вместо выдачи вероятности победы, модель будет выдавать вероятности для каждого возможного предмета, и вам нужно будет выбрать предмет с наибольшей вероятностью.\n\nДля этого, сначала вам нужно закодировать все возможные предметы в виде числовых индексов и добавить новый столбец 'next_item' в ваш датафрейм, который будет содержать индекс следующего предмета для каждой строки данных.\n\nПосле этого, вы можете использовать многослойную LSTM сеть для классификации следующего предмета. В качестве функции потерь вы можете использовать Cross Entropy Loss, т.к. это задача многоклассовой классификации.\n\nВот пример кода для создания и обучения модели:\n\n```python\nimport torch\nimport torch.nn as nn\nimport torch.optim as optim\n\n# Создаем LSTM модель\nclass LSTMModel(nn.Module):\n    def __init__(self, input_size, hidden_size, num_layers, output_size):\n        super(LSTMModel, self).__init__()\n        self.hidden_size = hidden_size\n        self.num_layers = num_layers\n        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)\n        self.fc = nn.Linear(hidden_size, output_size)\n\n    def forward(self, x):\n        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()\n        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()\n        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))\n        out = self.fc(out[:, -1, :])\n        return out\n\n# Гиперпараметры\ninput_size = data_tensor.shape[1]\nhidden_size = 64\nnum_layers = 2\noutput_size = num_unique_items  # Количество уникальных предметов\nnum_epochs = 100\nlearning_rate = 0.001\n\n# Инициализация модели, функции потерь и оптимизатора\nmodel = LSTMModel(input_size, hidden_size, num_layers, output_size)\ncriterion = nn.CrossEntropyLoss()\noptimizer = optim.Adam(model.parameters(), lr=learning_rate)\n\n# Обучение модели\nfor epoch in range(num_epochs):\n    # Преобразуем данные в нужную форму\n    inputs = data_tensor.unsqueeze(1)  # Добавляем размерность sequence_length\n    targets = torch.tensor(data['next_item'].values, dtype=torch.long)\n\n    # Forward pass\n    outputs = model(inputs)\n    loss = criterion(outputs, targets)\n\n    # Backward and optimize\n    optimizer.zero_grad()\n    loss.backward()\n    optimizer.step()\n\n    if (epoch + 1) % 10 == 0:\n        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')\n```\n\nПосле обучения модели, вы можете использовать ее для предсказания следующего предмета на основе текущих предметов, имени героя и его позиции. Просто передайте соответствующий тензор с данными в модель, и она выдаст вероятности для каждого предмета. Затем выберите предмет с наибольшей вероятностью.\n\n```python\nwith torch.no_grad():\n    input_data = torch.tensor([hero_name, lane] + items).unsqueeze(0).unsqueeze(1)\n    item_probabilities = torch.softmax(model(input_data), dim=1).squeeze()\n    recommended_item_index = torch.argmax(item_probabilities).item()\n    recommended_item = index_to_item[recommended_item_index]  # Предполагается, что у вас есть словарь index_to_item для преобразования индекса в предмет\n```\n\nОбратите внимание, что этот код является примером и может потребовать дополнительной настройки и оптимизации для вашей конкретной задачи. Вам также может потребоваться разделить ваши данные на обучающую и тестовую выборки для оценки производительности модели."]""")


      ['В python pandas я занимаюсь препроцессингом данных для модели lstm. В моих данных есть поле, содержащее списки одинакового размера, наполненные числами. Я преобразовал массивы в этом поле к numpy.array, и теперь когда я пытаюсь сделать нормализацию 
data_f_normalized = (data_f - data_f.min()) / (data_f.max() - data_f.min())
Мне выдает ошибку  The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()', "Ошибка возникает из-за того, что операции min() и max() возвращают массивы, а не скалярные значения, когда они применяются к столбцу с массивами numpy. Вам нужно найти минимальное и максимальное значения по всем элементам во всех массивах этого столбца.

Вот пример, как это можно сделать:

``` python
import pandas as pd
import numpy as np

# Создаем пример данных
data = pd.DataFrame({'A': [np.array([1, 2, 3]), np.array([4, 5, 6]), np.array([7, 8, 9])]})

# Находим минимальное и максимальное значения по всем элементам массивов
min_value = data['A